In [1]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os



Using TensorFlow backend.


In [ ]:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
activation_4 (Activation)    (None, 13, 13, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         

                 
                 
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 128)         73856     
_________________________________________________________________
activation_5 (Activation)    (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 128)         147584    
_________________________________________________________________
activation_6 (Activation)    (None, 4, 4, 128)         0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
=================================================================
Total params: 420,906
Trainable params: 420,906
Non-trainable params: 0
_________________________________________________________________


In [18]:
# model = Sequential()
inp = keras.models.Input(shape=(32,32,3))
x = Conv2D(32, (3, 3), padding='same', activation='relu')(inp)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.25)(x)

skip = x
#(None, 8, 8, 64) ==> (1, 1, 64)
channel = Conv2D(64, (8,8), padding='valid', activation='sigmoid')(x)
spatial = Conv2D(1, (1,1), padding='same', activation='sigmoid')(x)

channel_att = keras.layers.Lambda(lambda x: x[0]*x[1])([skip,channel])
spatial_att = keras.layers.Lambda(lambda x: x[0]*x[1])([skip,spatial])

x = keras.layers.concatenate([skip,channel_att, spatial_att])
##############################################################
# import keras.backend as K

# def myFunc(x):
#     return x[0] * x[1]

# x1 = keras.layers.GlobalAveragePooling2D()(x)
# x1 = model.add(Lambda(lambda x: x ** 2)) 
# cross1 = Lambda(myFunc, output_shape=....)([d1,d4])
##############################################################

x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.25)(x)

# x = Conv2D(32, (3, 3), padding='same', activation='relu')(inp)
# x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
# x = MaxPooling2D((2,2))(x)
# x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)

x = Dense(10, activation='linear')(x)

x = keras.layers.Activation('softmax')(x)
model = keras.Model(inputs=inp, outputs=x)


model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 32, 32, 32)   896         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 32, 32, 32)   9248        conv2d_33[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 16, 16, 32)   0           conv2d_34[0][0]                  
__________________________________________________________________________________________________
dropout_21

In [7]:
channel

<tf.Tensor 'conv2d_29/Sigmoid:0' shape=(?, 1, 1, 64) dtype=float32>

In [8]:
import keras.backend as k

In [11]:

k.repeat_elements(k.repeat_elements(channel,8,axis=1),8,axis=2)


<tf.Tensor 'concat_2:0' shape=(?, 8, 8, 64) dtype=float32>

In [13]:
keras.layers.Lambda(lambda x: x[0]*x[1])([skip,channel])

<tf.Tensor 'lambda_2/mul:0' shape=(?, 8, 8, 64) dtype=float32>

In [15]:
skip, spatial

(<tf.Tensor 'dropout_18/cond/Merge:0' shape=(?, 8, 8, 64) dtype=float32>,
 <tf.Tensor 'conv2d_30/Sigmoid:0' shape=(?, 8, 8, 1) dtype=float32>)

In [16]:
keras.layers.Lambda(lambda x: x[0]*x[1])([skip,spatial])

<tf.Tensor 'lambda_3/mul:0' shape=(?, 8, 8, 64) dtype=float32>

In [17]:
keras.layers.concatenate([skip,skip])

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 8, 8, 128) dtype=float32>